In [4]:
# load packages
from __future__ import print_function
import tensorflow as tf
from tensorflow.contrib import rnn
import numpy as np

C:\Users\xuhuag\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [ ]:
# First run load_data

In [ ]:
# Training hyperparameters
learning_rate = 0.01
training_steps = 10000
batch_size = 200
display_step = 200

In [ ]:
# Network hyperparameters
num_input = 10 # data input
timesteps = 2000 # timesteps
num_hidden = 120 # hidden layer num of features
num_Y = 1 # single number output

In [ ]:
# tf placeholder
X = tf.placeholder("float", [None, timesteps, num_input])
Y = tf.placeholder("float", [None, num_Y])

# Define parameters
weights = {
    'out': tf.Variable(tf.random_normal([2*num_hidden, num_Y]))
}
biases = {
    'out': tf.Variable(tf.random_normal([num_Y]))
}


# bidirectional RNN
def BiRNN(x, weights, biases):

    # Prepare data shape to match function requirements 
    x = tf.unstack(x, timesteps, 1)

    # Define lstm cells
    # Forward-prop cell
    lstm_fw_cell = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)
    # Backward-prop cell
    lstm_bw_cell = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)
    # output
    try:
        outputs, _, _ = rnn.static_bidirectional_rnn(lstm_fw_cell, lstm_bw_cell, x,
                                              dtype=tf.float32)
    except Exception:
        outputs = rnn.static_bidirectional_rnn(lstm_fw_cell, lstm_bw_cell, x,
                                        dtype=tf.float32)

    # Linear activation
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

prediction_temp = BiRNN(X, weights, biases)
prediction = tf.nn.tanh(prediction_temp)

# loss function and optimizer
loss_mse = tf.losses.mean_squared_error(
    labels=Y,
    prediction,
    scope=None,
    loss_collection=tf.GraphKeys.LOSSES,
    reduction=Reduction.SUM_BY_NONZERO_WEIGHTS
)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_mse)

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    for step in range(1, training_steps+1):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Reshape data to get 28 seq of 28 elements
        batch_x = batch_x.reshape((batch_size, timesteps, num_input))
        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_mse, accuracy], feed_dict={X: batch_x,
                                                                 Y: batch_y})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Optimization Finished!")

In [ ]:
trainScore = math.sqrt(mean_squared_error(trainY[:,0], trainPredict[:,0]))
testScore = math.sqrt(mean_squared_error(testY[:,0], testPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
print('Test Score: %.2f RMSE' % (testScore))